In [2]:
import pyvista
#!pip install pyvista
!pip install vtk

You should consider upgrading via the 'c:\users\catal\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [3]:
#%%
import vtk
from vtk.util.numpy_support import vtk_to_numpy
import numpy as np
import os 
import nibabel as nib

# Los landmarks del groun truth están en 'INRIA COORDINATES' 
# y 'VTK COORDINATES'. Las funciones 'read_inria_lmks' y 'read_vtk_lmks' 
# leen los landmarks en las coordenadas de INRIA y VTK respectivamente y 
# lss transforma a un pixel. Cambien los path según como tengan guardados 
# los datos. Al correrlo debieran obtener los mismos valores para las variables 
# 'lmks_gt_inria_0' y 'lmks_gt_vtk_0' que contienen las coordenadas en pixel. 
# Se asume una especie de espacio continuo en los pixels, por lo que uno no obtiene 
# números enteros al transformar los landmarks (lo que me generó la duda de 
# cómo trackearon los landmarks).


def read_inria_lmks(file_name, px, py, pz):
    """ Read INRIA Landmarks in mesh coordinates and 
    transform them to pixel coordinates"""

    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(file_name)
    reader.ReadAllScalarsOn()
    reader.ReadAllVectorsOn()
    reader.Update()

    polydata = reader.GetOutput()
    points = polydata.GetPoints()
    array = points.GetData()
    lmks_mesh = vtk_to_numpy(array)
    # Transform lmks coordinates to pixel coordinates
    lmks_pixel = mesh_to_pixel(lmks_mesh, px, py, pz)

    return lmks_pixel

def read_vtk_lmks(file_name, px, py, pz, b):
    """ Read VTK Landmarks in mesh coordinates and 
    transform them to pixel coordinates"""

    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(file_name)
    reader.ReadAllScalarsOn()
    reader.ReadAllVectorsOn()
    reader.Update()

    polydata = reader.GetOutput()
    points = polydata.GetPoints()
    array = points.GetData()
    # Next line transforms VTK coordinate system into INRIA coordinate system
    lmks_mesh = vtk_to_numpy(array) * np.array([-1,-1,1]) + b * np.array([-1, -1, -1])
    # Transform lmks coordinates to pixel coordinates
    lmks_pixel = mesh_to_pixel(lmks_mesh, px, py, pz)

    return lmks_pixel

def mesh_to_pixel(surf_mesh, px, py, pz):
    #print(f" {surf_mesh * np.array([-1/px, -1/py, 1/pz])}")
    return surf_mesh * np.array([-1/px, -1/py, 1/pz])
    
volunteer = 'v1'

# Load image of some volunteer at some time
data_path = os.path.join( 'results_v2', volunteer )
t1 = nib.load(os.path.join(data_path,'v1.nii.gz'))

# Extrac information from the image
header = t1.header

pix_dim = header['pixdim']
# Pixel spacing of images
pixsp_x = pix_dim[1]
pixsp_y = pix_dim[2]
pixsp_z = pix_dim[3]

A = t1.affine[:-1,:-1].astype(np.float32)
b = t1.affine[:-1,-1].astype(np.float32)

#%%
###

# Ground truth - INRIA COORDINATES

# Path to landmarks directory
inria_lmks_path = os.path.join('cMAC','cMAC','GT','SSFP', volunteer, 'LMKS/INRIA_COORDINATES')
# Path to file
file_name = os.path.join(inria_lmks_path, 'obs1_groundTruth11.vtk')
lmks_gt_inria_0 = read_inria_lmks(file_name, pixsp_x, pixsp_y, pixsp_z)

###punto incial

lmks_path = os.path.join('cMAC','cMAC','GT','SSFP', volunteer, 'LMKS/INRIA_COORDINATES')
# Path to file
file_name = os.path.join(lmks_path, 'obs1_groundTruth00.vtk')
lmks_gt_sub_0 = read_inria_lmks(file_name, pixsp_x, pixsp_y, pixsp_z)

# Ground truth - VTK COORDINATES

# Path to landmarks directory
vtk_lmks_path = os.path.join('cMAC','cMAC','GT','SSFP', volunteer, 'LMKS/VTK_COORDINATES')
# Path to file
file_name = os.path.join(vtk_lmks_path, 'obs1_groundTruth00.vtk')
lmks_gt_vtk_0 = read_vtk_lmks(file_name, pixsp_x, pixsp_y, pixsp_z, b)
# %%



In [4]:
import h5py
from scipy.interpolate import RegularGridInterpolator

In [5]:
##OFICIAL
from nilearn.image.image import load_img
import meshio
lista = [[10,10], [22,29]] 
frames = {"v1": [[10,10], [22,29]], "v2": [[10,11], [28,29]] , "v4": [[10,11], [25,29]] ,\
     "v5": [[10,11], [22,29]] , "v6": [[10,11], [22,29]], "v7": [[11,9], [30,29]],\
      "v8": [[10,9], [30,29]], "v9": [[10,10], [29,29]], "v10": [[10,10], [26,29]],\
           "v11": [[11,9], [31,29]], "v12": [[10,11], [23,29]], "v13": [[10,9], [37,29]], \
               "v14": [[11,10], [28,29]], "v15": [[8,11], [20,29]], "v16": [[9,11], [24,29]]}
diccionario = {}
diccionario_medianas = {}
for volunteer, values in frames.items():
    data_path = os.path.join( 'results_v2', volunteer )
    t1 = nib.load(os.path.join(data_path,f'{volunteer}.nii.gz'))

    # Extrac information from the image
    header = t1.header

    pix_dim = header['pixdim']
    # Pixel spacing of images
    pixsp_x = pix_dim[1]
    pixsp_y = pix_dim[2]
    pixsp_z = pix_dim[3]
    #print(volunteer)
    #print(values)
    lista_errores = []
    lista_puntos = []
    for i_landmark, i_cine in values:
        lmks_path = os.path.join('cMAC','cMAC','GT','SSFP', volunteer, 'LMKS/INRIA_COORDINATES')
        file_name = os.path.join(lmks_path, "obs1_groundTruth%.2i.vtk" % i_landmark)
        lmks_gt_sub_0 = read_inria_lmks(file_name, pixsp_x, pixsp_y, pixsp_z)
        #cambiar variable file pot otro nombre
        file = h5py.File(f'./results/{volunteer}/{volunteer}_motion.h5', 'r')
        Nx, Ny, Nz, _ = file[f"frame_%.1i" % i_cine]["u"].shape
        
        x = np.arange(Nx)
        y = np.arange(Ny)
        z = np.arange(Nz)

        u1_pred = RegularGridInterpolator((x,y,z),  file[f"frame_%.1i" % i_cine]["u"][:,:,:,])
        u2_pred = RegularGridInterpolator((x,y,z),  file[f"frame_%.1i" % i_cine]["u"][:,:,:,])
        u3_pred = RegularGridInterpolator((x,y,z),  file[f"frame_%.1i" % i_cine]["u"][:,:,:,])
        
        for point in lmks_gt_sub_0: 
            nx = point[0]
            ny = point[1]
            nz = point[2]
            u_pred = np.array([u1_pred(np.array([nx, ny, nz])), u2_pred(np.array([nx, ny, nz])), u3_pred(np.array([nx, ny, nz]))])
            pred_pos = u_pred + np.array([nx,ny,nz])
            #pred_pos = file[f"frame_%.1i" % i_cine]["u"][nx,ny,nz] + np.array([nx,ny,nz])
            #print(pred_pos[0])
            v = (point - pred_pos)*np.array([pixsp_x, pixsp_y, pixsp_z])
            error = np.linalg.norm(v)
            lista_errores.append(error)
            lista_puntos.append([pred_pos*np.array([-pixsp_x, -pixsp_y, pixsp_z])])

        meshio.write(f'./results/{volunteer}/predlandmarks_{volunteer}_{i_cine}.vtk', meshio.Mesh(points=np.array(lista_puntos), cells={}))

    mediana = np.median(lista_errores)
    diccionario_medianas[volunteer] = mediana
    diccionario[volunteer] = lista_errores
print(diccionario_medianas)
print(len(diccionario))


{'v1': 6.260595380629347, 'v2': 7.099822728008526, 'v4': 6.916853808005037, 'v5': 10.817963095759557, 'v6': 8.437695331136993, 'v7': 3.4722889665037915, 'v8': 2.3700487765400813, 'v9': 1.9200407559963288, 'v10': 2.7158678038206054, 'v11': 1.2712867716935503, 'v12': 4.330864268696716, 'v13': 1.2402037287685412, 'v14': 0.22783559605529463, 'v15': 4.308795221882262, 'v16': 7.342425544362921}
15


In [6]:
np.savez("errores.npz",**diccionario)